In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("hhw_langchain_toy_project")

LangSmith 추적을 시작합니다.
[프로젝트명]
hhw_langchain_toy_project


In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template("{num} 의 10배는?")

# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="gemma3:1b")

# chain 을 생성합니다.
chain = prompt | llm

/var/folders/5d/4yvbq7s16359fdr947szsx5h0000gn/T/ipykernel_37703/1800357322.py:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="gemma3:1b")


In [4]:
# chain 을 실행합니다.
chain.invoke({"num": 5})

AIMessage(content='5는 10배가 아닙니다. 5는 10배가 아닙니다. 5는 2배입니다.', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-03-23T10:33:30.207100094Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6474705545, 'load_duration': 3335239710, 'prompt_eval_count': 17, 'prompt_eval_duration': 1033652125, 'eval_count': 30, 'eval_duration': 2097683084}, id='run-797adbe7-0a89-4504-94db-8818d351a046-0')

In [5]:
# chain 을 실행합니다.
chain.invoke(5)

AIMessage(content='5는 10배가 아닙니다. 5는 2배입니다.\n\n5의 10배는 10입니다.', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-03-23T10:33:50.213250089Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2814156376, 'load_duration': 89688208, 'prompt_eval_count': 17, 'prompt_eval_duration': 441230667, 'eval_count': 32, 'eval_duration': 2280109251}, id='run-19e398a3-5ef7-487f-bee5-71a979d71d8a-0')

In [6]:
from langchain_core.runnables import RunnablePassthrough

# runnable
RunnablePassthrough().invoke({"num": 10})

{'num': 10}

In [8]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | llm

# dict 값이 RunnablePassthrough() 로 변경되었습니다.
runnable_chain.invoke(10)

AIMessage(content='10의 10배는 100입니다.\n\n계산 방법: 10 * 10 = 100', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-03-23T10:35:13.595275711Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2931879877, 'load_duration': 97816834, 'prompt_eval_count': 18, 'prompt_eval_duration': 931994542, 'eval_count': 32, 'eval_duration': 1899795917}, id='run-6eb6592e-3f4b-4929-ab03-caf669fb11d3-0')

In [9]:
RunnablePassthrough().invoke({"num": 1})

{'num': 1}

In [10]:
# 입력 키: num, 할당(assign) 키: new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})


{'num': 1, 'new_num': 3}

In [11]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스를 생성합니다. 이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행할 수 있습니다.
runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달합니다. 이는 입력된 데이터를 그대로 통과시키는 역할을 합니다.
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용하여, 'mult' 람다 함수를 할당합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가시킵니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더합니다.
    modified=lambda x: x["num"] + 1,
)

# runnable 인스턴스에 {'num': 1} 딕셔너리를 입력으로 전달하여 invoke 메소드를 호출합니다.
runnable.invoke({"num": 1})


{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [12]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | llm
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | llm
)


In [13]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='대한민국 의 수도는 서울입니다.', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-03-23T10:37:10.709611001Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2342013168, 'load_duration': 89916250, 'prompt_eval_count': 15, 'prompt_eval_duration': 1236112709, 'eval_count': 9, 'eval_duration': 1013646584}, id='run-474dbf65-5afa-4fc3-982e-fc2e04b8b97c-0'),
 'area': AIMessage(content='대한민국의 면적은 약 108,334 제곱킬로미터입니다. 이는 지리적으로는 세계에서 세 번째로 큰 나라입니다.\n\n*   **면적:** 약 637,870 제곱킬로미터\n*   **지구 표면 면적:** 약 98,260 제곱킬로미터 (대한민국 전체 면적의 약 31%)\n\n참고로, 대한민국은 18도 너비의 18개도를 포함하고 있습니다.', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-03-23T10:37:17.368019713Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 9000561629, 'load_duration': 90972333, 'prompt_eval_count': 16, 'prompt_eval_duration': 1234665626, 'eval

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from datetime import datetime


def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)


'Mar-23'

In [15]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [16]:
# 출력
print(chain.invoke(3))

Mar-23 가 생일인 유명인 3 명은 다음과 같습니다.

*   **마르코 로브리츠키** (1975년 1월 31일) - 미국 배우
*   **에밀리 블런트** (1981년 2월 3일) - 미국 배우
*   **마이클 잭슨** (1958년 12월 8일) - 미국 가수, 배우

이 정보는 현재까지 제가 알고 있는 정보를 기반으로 합니다.


In [18]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


# 문장의 길이를 반환하는 함수입니다.
def length_function(text):
    return len(text)


# 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")

chain1 = prompt | llm

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | llm
)


In [19]:
chain.invoke({"word1": "hello", "word2": "world"})


AIMessage(content='5 + 25 = 30\n\n5 + 25는 30입니다.', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-03-23T10:38:47.873862255Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2054727584, 'load_duration': 76504792, 'prompt_eval_count': 19, 'prompt_eval_duration': 811752459, 'eval_count': 22, 'eval_duration': 1163836084}, id='run-fbd1f050-bf3d-484c-9985-9d2975abb27b-0')